In [37]:
!pip install datasets tensorflow keras

In [38]:
import numpy as np
import tensorflow as tf
from datasets import load_dataset
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import re

In [39]:
# ---------------------------------------------------------
# 1. LOAD IMDB RAW TEXT
# ---------------------------------------------------------
dataset = load_dataset("imdb")

# Clean training text
texts = [
    re.sub(r"<br\s*/?>", " ", t.lower())  # remove HTML + lowercase
    for t in dataset["train"]["text"]
]
labels = np.array(dataset["train"]["label"])

# Clean test text
test_texts = [
    re.sub(r"<br\s*/?>", " ", t.lower())
    for t in dataset["test"]["text"]
]
test_labels = np.array(dataset["test"]["label"])

print("Dataset loaded.")
print("Training samples:", len(texts))
print("Testing samples:", len(test_texts))

Dataset loaded.
Training samples: 25000
Testing samples: 25000


In [40]:
# ---------------------------------------------------------
# 2. TOKENIZE
# ---------------------------------------------------------
vocab_size = 30000
max_length = 250

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

X = tokenizer.texts_to_sequences(texts)
X_test = tokenizer.texts_to_sequences(test_texts)

X = pad_sequences(X, maxlen=max_length, padding="post")
X_test = pad_sequences(X_test, maxlen=max_length, padding="post")

In [41]:
# ---------------------------------------------------------
# 3. SHUFFLE & SPLIT TRAIN/VAL (THIS FIXES THE ISSUE)
# ---------------------------------------------------------
X_train, X_val, y_train, y_val = train_test_split(
    X, labels, test_size=0.2, random_state=42, shuffle=True
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

Train: (20000, 250) Val: (5000, 250) Test: (25000, 250)


In [42]:
# ---------------------------------------------------------
# 4. HIGH-ACCURACY STACKED LSTM MODEL
# ---------------------------------------------------------
embedding_dim = 128

model = Sequential([
    Input(shape=(max_length,)),
    Embedding(vocab_size, embedding_dim),

    # FIRST LSTM
    LSTM(128, dropout=0.3),

    Dense(64, activation='relu'),
    Dropout(0.3),

    Dense(1, activation='sigmoid')   # binary
])


model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 250, 128)       │     3,840,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,979,905 (15.18 MB)

 Trainable params: 3,979,905 (15.18 MB)

 Non-trainable params: 0 (0.00 B)

In [43]:
# ---------------------------------------------------------
# 5. TRAIN WITH EARLY STOPPING
# ---------------------------------------------------------
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=64,
    callbacks=[early_stop]
)

Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.5279 - loss: 0.6860 - val_accuracy: 0.5780 - val_loss: 0.6390
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.6032 - loss: 0.6000 - val_accuracy: 0.5850 - val_loss: 0.6286
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.6258 - loss: 0.5445 - val_accuracy: 0.6066 - val_loss: 0.6375
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.7910 - loss: 0.4114 - val_accuracy: 0.8690 - val_loss: 0.3795
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.9280 - loss: 0.2053 - val_accuracy: 0.8676 - val_loss: 0.4184
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.9624 - loss: 0.1261 - val_accuracy: 0.8700 - val_loss: 0.4260


In [44]:
# ---------------------------------------------------------
# 5. EVALUATE ON TEST DATASET
# ---------------------------------------------------------
loss, acc = model.evaluate(X_test, y_test)
print(f"\n🔥 LSTM IMDB RAW TEXT Test Accuracy: {acc*100:.2f}%")

782/782 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8757 - loss: 0.3186

🔥 LSTM IMDB RAW TEXT Test Accuracy: 85.21%


In [46]:
def clean_for_pred(text):
    text = text.lower()
    text = re.sub(r"<br\s*/?>", " ", text)
    text = re.sub(r"[^a-zA-Z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def predict(text):
    text = clean_for_pred(text)
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=max_length, padding="post")
    prob = model.predict(pad)[0][0]
    label = "Positive" if prob > 0.5 else "Negative"
    return label, prob

print("\n🧪 Custom Predictions:")
samples = [
    "I absolutely loved this movie! One of the best.",
    "This was terrible. I hated every moment.",
    "It was okay, not too bad, but not good either.",

    # 🔥 Strong Positive
    "What a fantastic film! I would watch it again.",
    "Amazing acting and a beautiful storyline.",
    "Loved every second of it, truly a masterpiece.",

    # ❄️ Strong Negative
    "One of the worst movies ever made. Total waste of time.",
    "Horrible acting, terrible script, zero entertainment value.",
    "I regret watching this, it was painfully bad.",

    # 🙂 Mild Positive
    "It was enjoyable and better than I expected.",
    "Not amazing, but I liked it overall.",
    "Pretty good movie, decent plot and good acting.",

    # 😐 Neutral / Mixed
    "Some parts were great, others were boring.",
    "The movie was fine, nothing special though.",
    "It was an average film with a predictable storyline.",

    # 😒 Mild Negative
    "The movie wasn’t terrible, but it was definitely disappointing.",
    "It dragged too much and felt really slow.",
    "Not the worst, but I wouldn’t recommend it.",

    # 🌀 Ambiguous / Hard to classify
    "I don’t know what to feel about this movie.",
    "Good visuals but weak story.",
    "The movie started strong but fell apart by the end.",

    # 😂 Sarcastic (hard but good test)
    "Oh wow, what a brilliant movie… said no one ever.",
    "Great… another movie that put me to sleep."
]

for s in samples:
    label, p = predict(s)
    print(f"{s}\n→ {label} ({p:.4f})\n")


🧪 Custom Predictions:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
I absolutely loved this movie! One of the best.
→ Positive (0.8549)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
This was terrible. I hated every moment.
→ Negative (0.0945)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
It was okay, not too bad, but not good either.
→ Negative (0.0764)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
What a fantastic film! I would watch it again.
→ Positive (0.7908)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Amazing acting and a beautiful storyline.
→ Positive (0.6976)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Loved every second of it, truly a masterpiece.
→ Positive (0.8480)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
One of the worst movies ever made. Total waste of time.
→ Negative (0.0565)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Horrible acting, terrible script, zero entertainment value.
→ Negative (0.0578)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
I regret watching this, it was painfully bad.
→ Negative (0.1410)

1/1 ━━━━━━━━